# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [7]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import json

with open("Sample_Notebook.ipynb", "r", encoding="utf-8") as f:
    data = json.load(f)

# Clean widget metadata
if "widgets" in data.get("metadata", {}):
    data["metadata"]["widgets"]["state"] = {}

# Save cleaned notebook
with open("your_notebook_cleaned.ipynb", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

print("✅ Notebook cleaned! Try opening the new file.")


FileNotFoundError: [Errno 2] No such file or directory: 'your_notebook.ipynb'

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [20]:
import pandas as pd

# Load the already processed dataset
df = pd.read_csv("dataset.csv")

# --- Step 1: Ensure column names are clean ---
df.columns = df.columns.str.strip()

# --- Step 2: Ensure Timestamp is a datetime object ---
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors='coerce')
df = df.dropna(subset=["Timestamp"])  # Remove invalid timestamps

# --- Step 3: Confirm necessary columns exist ---
required_columns = [
    "ID", "SystemCodeNumber", "Capacity", "Latitude", "Longitude", "Occupancy",
    "VehicleType", "TrafficConditionNearby", "QueueLength",
    "IsSpecialDay", "Timestamp"
]
missing = set(required_columns) - set(df.columns)
if missing:
    raise ValueError(f"Dataset is missing these columns: {missing}")

# --- Step 4: Map traffic condition to score ---
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df["TrafficScore"] = df["TrafficConditionNearby"].map(traffic_map)

# --- Step 5: Map vehicle type to weight ---
vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5, 'cycle': 0.3}
df["VehicleWeight"] = df["VehicleType"].map(vehicle_weights)

# --- Step 6: Drop any rows with missing mapped values ---
df = df.dropna(subset=["TrafficScore", "VehicleWeight"])

# --- Step 7: Sort by timestamp ---
df = df.sort_values("Timestamp").reset_index(drop=True)

# --- Step 8: Reorder columns (optional) ---
df = df[[
    "ID", "SystemCodeNumber", "Capacity", "Latitude", "Longitude", "Occupancy",
    "VehicleType", "VehicleWeight", "TrafficConditionNearby", "TrafficScore",
    "QueueLength", "IsSpecialDay", "Timestamp"
]]

# --- Step 9: Save the cleaned file ---
df.to_csv("dataset_cleaned.csv", index=False)

print("✅ Preprocessing successful! Saved as 'dataset_cleaned.csv'")
print(f"Total records: {len(df)}, Unique Parking Lots: {df['SystemCodeNumber'].nunique()}")


✅ Preprocessing successful! Saved as 'dataset_cleaned.csv'
Total records: 18368, Unique Parking Lots: 14


In [21]:
import pandas as pd

df = pd.read_csv("dataset.csv")

# ✅ Step 1: Combine date and time into Timestamp only if needed
if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
    df['Timestamp'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        format='%d-%m-%Y %H:%M:%S'
    )
    df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime'], inplace=True)
elif 'Timestamp' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # already present
else:
    raise ValueError("No date/time columns found to generate Timestamp.")

# ✅ Step 2: Sort and reset index
df = df.sort_values('Timestamp').reset_index(drop=True)

print("✅ Timestamp column is ready and sorted.")


✅ Timestamp column is ready and sorted.


In [23]:
import pandas as pd

# === Step 1: Load the already parsed dataset ===
df = pd.read_csv("dataset.csv")

# === Step 2: Clean column names (if needed) ===
df.columns = df.columns.str.strip()

# ✅ Check presence of expected columns (excluding date/time)
required_columns = {
    'ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
    'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
    'IsSpecialDay', 'Timestamp'
}
missing = required_columns - set(df.columns)
if missing:
    raise ValueError(f"❌ Missing columns in dataset: {missing}")

# === Step 3: Convert Timestamp column to datetime ===
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df = df.dropna(subset=['Timestamp'])

# === Step 4: Map traffic conditions to scores ===
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficScore'] = df['TrafficConditionNearby'].map(traffic_map)

# === Step 5: Map vehicle types to numeric weights ===
vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5, 'cycle': 0.3}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

# === Step 6: Sort chronologically for streaming ===
df = df.sort_values("Timestamp").reset_index(drop=True)

# === Step 7: Save files ===

# Full cleaned dataset
df.to_csv("dataset_cleaned.csv", index=False)
print("✅ Full cleaned dataset saved to 'dataset_cleaned.csv'")

# Minimal for Model 1
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)
print("✅ Minimal stream file saved as 'parking_stream.csv'")

# Feature-rich for advanced models
df[[
    "Timestamp", "SystemCodeNumber", "Occupancy", "Capacity",
    "QueueLength", "TrafficScore", "IsSpecialDay",
    "VehicleType", "VehicleWeight"
]].to_csv("parking_stream_full.csv", index=False)
print("✅ Full-feature stream file saved as 'parking_stream_full.csv'")


✅ Full cleaned dataset saved to 'dataset_cleaned.csv'
✅ Minimal stream file saved as 'parking_stream.csv'
✅ Full-feature stream file saved as 'parking_stream_full.csv'


In [28]:
import pathway as pw
import pathway.internals.dtype as dtype

# === Step 1: Define Schema ===
class ParkingSchema(pw.Schema):
    Timestamp: dtype.DATE_TIME_UTC
    Occupancy: int
    Capacity: int

# === Step 2: Read data stream from folder ===
input_table = pw.io.csv.read(
    "./input_data",        # folder where streamed CSVs will appear
    schema=ParkingSchema,  # your schema defined above
    mode="streaming",      # enables real-time simulation
    autocommit_duration_ms=1000  # adjust how often to simulate updates
)

# === Step 3: Compute occupancy ratio and linear price ===
# Base pricing parameters
BASE_PRICE = 10.0
ALPHA = 5.0     # price sensitivity to occupancy
MIN_PRICE = 5.0
MAX_PRICE = 20.0

@pw.udf
def compute_price(occupancy: int, capacity: int) -> float:
    ratio = occupancy / capacity if capacity else 0
    price = BASE_PRICE + ALPHA * ratio
    return max(MIN_PRICE, min(MAX_PRICE, price))

# Add new column with computed prices
output_table = input_table.select(
    Timestamp=input_table.Timestamp,
    Occupancy=input_table.Occupancy,
    Capacity=input_table.Capacity,
    Price=compute_price(input_table.Occupancy, input_table.Capacity)
)

# === Step 4: Output continuously to a JSON file ===
pw.io.jsonlines.write(output_table, "output_stream.jsonl")

# === Step 5: Run ===
pw.run()


Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


KeyboardInterrupt: 

In [35]:
import pandas as pd

# Load raw or cleaned dataset
df = pd.read_csv("dataset.csv")

# Ensure Timestamp column is properly parsed and in ISO format
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
df = df.dropna(subset=["Timestamp"])  # Remove bad timestamps if any

# Convert to UTC ISO 8601 format (Pathway expects this)
df["Timestamp"] = df["Timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Save only required columns
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

print("✅ Saved 'parking_stream.csv' in correct ISO datetime format.")


import pathway as pw
import pathway.internals.dtype as dtype

# === Step 1: Define Schema ===
class ParkingSchema(pw.Schema):
    Timestamp: dtype.DATE_TIME_UTC
    Occupancy: int
    Capacity: int

# === Step 2: Simulate streaming from CSV (timestamp is auto-detected from schema)
input_table = pw.demo.replay_csv(
    "parking_stream.csv",      # Your CSV file
    schema=ParkingSchema,      # Use correct Pathway schema
    input_rate=100             # rows per second
)

# === Step 3: Calculate occupancy ratio with clipping
occupancy_raw = input_table.Occupancy / input_table.Capacity
input_table = input_table.with_columns(
    occupancy_ratio=pw.if_else(
        occupancy_raw > 1.0, 1.0,
        pw.if_else(occupancy_raw < 0.0, 0.0, occupancy_raw)
    )
)

# === Step 4: Dynamic pricing model (Model 1)
base_price = 10.0
alpha = 10.0

input_table = input_table.with_columns(
    price=(base_price + alpha * input_table.occupancy_ratio)
)

# === Step 5: Output results
pw.io.jsonlines.write(input_table.select(
    input_table.Timestamp,
    input_table.Occupancy,
    input_table.Capacity,
    input_table.occupancy_ratio,
    input_table.price
), "output_stream.jsonl")

# === Step 6: Run the streaming pipeline
pw.run()


✅ Saved 'parking_stream.csv' in correct ISO datetime format.


Output()

TypeError: Pathway doesn't support casting STR to DATE_TIME_UTC.

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [36]:
import pathway as pw
import pathway.internals.dtype as dtype
import datetime

# === Step 1: Define schema ===
class ParkingSchema(pw.Schema):
    Timestamp: dtype.DATE_TIME_UTC
    Occupancy: int
    Capacity: int

# === Step 2: Load simulated stream ===
input_table = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=100
)

# === Step 3: Extract event time and calendar day ===
data_with_time = input_table.with_columns(
    t=input_table.Timestamp,
    day=input_table.Timestamp.dt.date()
)

# === Step 4: Daily tumbling window aggregation ===
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# === Step 5: Output daily prices to JSONL ===
pw.io.jsonlines.write(
    delta_window.select(
        delta_window.t,
        delta_window.occ_max,
        delta_window.occ_min,
        delta_window.cap,
        delta_window.price
    ),
    "daily_dynamic_prices.jsonl"
)

# === Step 6: Run pipeline ===
pw.run()


AttributeError: 'DateTimeNamespace' object has no attribute 'date'

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [38]:
# === Step 0: Imports ===
import datetime
import pathway as pw
import pathway.internals.dtype as dtype
import bokeh.plotting
import panel as pn

# === Step 1: Define Schema ===
class ParkingSchema(pw.Schema):
    Timestamp: dtype.DATE_TIME_UTC
    Occupancy: int
    Capacity: int

# === Step 2: Simulate stream from CSV ===
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=100  # rows per second
)

# === Step 3: Extract day from timestamp for partitioning ===
data_with_day = data.with_columns(
    day=pw.apply(lambda ts: ts.date(), data.Timestamp),  # Extract date for windowing
    t=data.Timestamp
)

# === Step 4: Tumbling window aggregation ===
delta_window = (
    data_with_day.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# === Step 5: Bokeh + Panel Interactive Plot ===
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="🚗 Daily Parking Price Over Time",
        x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset,save",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")

    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Price"
    fig.grid.grid_line_alpha = 0.3
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

# === Step 6: Make visual dashboard servable ===
pn.Column(
    "# 📊 Pathway Dashboard: Dynamic Parking Prices",
    "This dashboard shows day-wise dynamic prices based on parking occupancy trends.",
    viz
).servable()


Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()